# Predicting Customer Churn in the Telecom Sector
This notebook analyzes the Telco Customer Churn dataset to build models for predicting customer churn.
The dataset is sourced from Kaggle: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
# Make sure the 'Telco-Customer-Churn.csv' file is in the same directory as this notebook.
data = pd.read_csv('Telco-Customer-Churn.csv')

# Display the first few rows of the dataset
data.head()

## Data Preprocessing
In this section, we clean the data, encode categorical variables, and prepare it for modeling.

In [ ]:
# Step 1: Handle missing values
# Drop rows with missing values to ensure data quality
data = data.dropna()

# Step 2: Encode categorical variables
# Use LabelEncoder to convert categorical features into numeric values
label_encoders = {}  # Dictionary to store encoders for potential inverse transformations
for column in data.select_dtypes(include=['object']).columns:
    if column != 'Churn':  # Skip the target variable for now
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

# Encode the target variable ('Churn')
# Convert 'Yes' to 1 and 'No' to 0
data['Churn'] = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Step 3: Separate features and target variable
X = data.drop('Churn', axis=1)  # Features
y = data['Churn']             # Target variable

# Step 4: Standardize numerical features
# Use StandardScaler to normalize features for better model performance
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the training and testing sets
X_train.shape, X_test.shape

## Model Training and Evaluation
In this section, we train a Random Forest Classifier and evaluate its performance.

In [ ]:
# Step 1: Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)  # Initialize the model
model.fit(X_train, y_train)  # Train the model

# Step 2: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 3: Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:
', classification_report(y_test, y_pred))

# Step 4: Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Feature Importance
Identify the most important features influencing customer churn.

In [ ]:
# Step 5: Feature importance analysis
# Extract feature importance scores from the Random Forest model
feature_importance = model.feature_importances_
feature_names = data.drop('Churn', axis=1).columns

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot the top 10 most important features
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(10), palette='viridis')
plt.title('Top 10 Important Features')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()